In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder,OneHotEncoder,MinMaxScaler,StandardScaler

from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
import seaborn as sbn

train_data=pd.read_csv('train_data.csv')
train_data['type']='train'
test=pd.read_csv('test_a.csv')
test['tradeMoney']=0
test['type']='test'

real_data=pd.read_csv('sub_a_913.csv')

In [2]:
#删除异常数据

#删除结合区域面积和交易金额的异常数据
#根据上述图，我们可以将一定范围内的异常点进行清除
def deep_delete(data):
    print("删除前：",data.shape[0])
    #删除tradeMoney等于0的数据
    data.drop(data[(data['tradeMoney']==0)&(data['type']=='train')].index,inplace=True)
    print("交易金额等于0的删除后{}".format(data.shape[0]))
    data.drop(data[(data['region']=='RG00001')&((data['tradeMoney']>50000)|(data['area']>200))].index,inplace=True)
    data.drop(data[(data['region']=='RG00002')&((data['tradeMoney']>100000)|(data['area']>500))].index,inplace=True)
    data.drop(data[(data['region']=='RG00003')&((data['tradeMoney']>50000)|(data['area']>400))].index,inplace=True)
    data.drop(data[(data['region']=='RG00007')&((data['tradeMoney']>50000)|(data['area']>200))].index,inplace=True)
    data.drop(data[(data['region']=='RG00010')&((data['tradeMoney']>50000)|(data['area']>500))].index,inplace=True)
    data.drop(data[(data['region']=='RG00008')&((data['tradeMoney']>50000)|(data['area']>200))].index,inplace=True)
    data.drop(data[(data['region']=='RG00004')&((data['tradeMoney']>30000)|(data['area']>250))].index,inplace=True)
    data.drop(data[(data['region']=='RG00012')&((data['tradeMoney']>50000)|(data['area']>200))].index,inplace=True)
    data.drop(data[(data['region']=='RG00009')&((data['tradeMoney']>25000)|(data['area']>200))].index,inplace=True)
    data.drop(data[(data['region']=='RG00011')&((data['tradeMoney']>40000)|(data['area']>500))].index,inplace=True)
    data.drop(data[(data['region']=='RG00013')&((data['tradeMoney']>40000)|(data['area']>250))].index,inplace=True)
    data.drop(data[(data['region']=='RG00014')&((data['tradeMoney']>20000)|(data['area']>250))].index,inplace=True)
    data.drop(data[(data['region']=='RG00005')&((data['tradeMoney']>50000)|(data['area']>400))].index,inplace=True)
    return data

train_data=deep_delete(train_data)
print("区域面积金额删除后{}".format(train_data.shape[0]))

删除前： 41440
交易金额等于0的删除后41431
区域面积金额删除后41327


In [3]:
#提取每平方均价特征
#测试集按小区的平均值进行构建
'''
train_data['ave_money']=train_data['tradeMoney']/train_data['area']
train_d=train_data.groupby(['communityName'])[['ave_money']].mean().reset_index()
test=test.merge(train_d,on=['communityName'],how='left')
'''

"\ntrain_data['ave_money']=train_data['tradeMoney']/train_data['area']\ntrain_d=train_data.groupby(['communityName'])[['ave_money']].mean().reset_index()\ntest=test.merge(train_d,on=['communityName'],how='left')\n"

In [4]:
data=pd.concat([train_data,test],axis=0)
data

,ID,area,rentType,houseType,houseFloor,totalFloor,houseToward,houseDecoration,communityName,city,...,landMeanPrice,totalWorkers,newWorkers,residentPopulation,pv,uv,lookNum,tradeTime,tradeMoney,type
0,100309852,68.06,未知方式,2室1厅1卫,低,16,暂无数据,其他,XQ00051,SH,...,0.000000,28248,614,111546,1124.0,284.0,0,2018/11/28,2000.0,train
1,100307942,125.55,未知方式,3室2厅2卫,中,14,暂无数据,简装,XQ00130,SH,...,0.000000,14823,148,157552,701.0,22.0,1,2018/12/16,2000.0,train
2,100307764,132.00,未知方式,3室2厅2卫,低,32,暂无数据,其他,XQ00179,SH,...,0.000000,77645,520,131744,57.0,20.0,1,2018/12/22,16000.0,train
3,100306518,57.00,未知方式,1室1厅1卫,中,17,暂无数据,精装,XQ00313,SH,...,3080.033100,8750,1665,253337,888.0,279.0,9,2018/12/21,1600.0,train
4,100305262,129.00,未知方式,3室2厅3卫,低,2,暂无数据,毛坯,XQ01257,SH,...,0.000000,800,117,125309,2038.0,480.0,0,2018/11/18,2900.0,train
5,100304268,223.35,未知方式,4室3厅3卫,低,2,暂无数据,其他,XQ00457,SH,...,0.000000,48204,44561,719428,4832.0,1304.0,0,2018/11/14,14000.0,train
6,100304055,78.94,未知方式,2室2厅1卫,中,18,暂无数据,其他,XQ00246,SH,...,0.000000,8750,1467,253337,1586.0,469.0,0,2018/11/18,1000.0,train
7,100295123,76.00,未知方式,2室1厅1卫,中,5,暂无数据,其他,XQ02331,SH,...,0.000000,28546,0,371900,40787.0,2942.0,0,2018/3/24,1800.0,train
8,100293141,120.55,未知方式,3室2厅1卫,中,6,暂无数据,其他,XQ02346,SH,...,0.000000,28546,0,371900,40787.0,2942.0,0,2018/3/19,1450.0,train
9,100293063,83.00,未知方式,3室1厅1卫,中,5,暂无数据,其他,XQ02310,SH,...,0.000000,28546,0,371900,34441.0,3035.0,0,2018/4/26,1700.0,train


In [5]:
# tradeTime 可以分为年月日,因为都是18年的数据，所以提取月日数据特征就好
# pv ，uv 先用均值填充
# 将"rentType"缺失值用众数进行填充
data['rentType'][data['rentType']=='--']='未知方式'

def months(d):
    s=d.split('/')
    return int(s[1])

def days(d):
    s=d.split('/')
    return int(s[2])
data['month']=data['tradeTime'].apply(months)
data['days']=data['tradeTime'].apply(days)

data['pv']=data['pv'].fillna(data[data['pv'].notnull()]['pv'].mean())
data['uv']=data['uv'].fillna(data[data['uv'].notnull()]['uv'].mean())
data[data['pv'].notnull()]

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,ID,area,rentType,houseType,houseFloor,totalFloor,houseToward,houseDecoration,communityName,city,...,newWorkers,residentPopulation,pv,uv,lookNum,tradeTime,tradeMoney,type,month,days
0,100309852,68.06,未知方式,2室1厅1卫,低,16,暂无数据,其他,XQ00051,SH,...,614,111546,1124.0,284.0,0,2018/11/28,2000.0,train,11,28
1,100307942,125.55,未知方式,3室2厅2卫,中,14,暂无数据,简装,XQ00130,SH,...,148,157552,701.0,22.0,1,2018/12/16,2000.0,train,12,16
2,100307764,132.00,未知方式,3室2厅2卫,低,32,暂无数据,其他,XQ00179,SH,...,520,131744,57.0,20.0,1,2018/12/22,16000.0,train,12,22
3,100306518,57.00,未知方式,1室1厅1卫,中,17,暂无数据,精装,XQ00313,SH,...,1665,253337,888.0,279.0,9,2018/12/21,1600.0,train,12,21
4,100305262,129.00,未知方式,3室2厅3卫,低,2,暂无数据,毛坯,XQ01257,SH,...,117,125309,2038.0,480.0,0,2018/11/18,2900.0,train,11,18
5,100304268,223.35,未知方式,4室3厅3卫,低,2,暂无数据,其他,XQ00457,SH,...,44561,719428,4832.0,1304.0,0,2018/11/14,14000.0,train,11,14
6,100304055,78.94,未知方式,2室2厅1卫,中,18,暂无数据,其他,XQ00246,SH,...,1467,253337,1586.0,469.0,0,2018/11/18,1000.0,train,11,18
7,100295123,76.00,未知方式,2室1厅1卫,中,5,暂无数据,其他,XQ02331,SH,...,0,371900,40787.0,2942.0,0,2018/3/24,1800.0,train,3,24
8,100293141,120.55,未知方式,3室2厅1卫,中,6,暂无数据,其他,XQ02346,SH,...,0,371900,40787.0,2942.0,0,2018/3/19,1450.0,train,3,19
9,100293063,83.00,未知方式,3室1厅1卫,中,5,暂无数据,其他,XQ02310,SH,...,0,371900,34441.0,3035.0,0,2018/4/26,1700.0,train,4,26


In [6]:
#提取房间数量特征
def room_num(room):
    return int(room[0])

data['room_num']=data['houseType'].apply(room_num)

#提取大厅数量特征
def lobby_num(lobby):
    return int(lobby[2])

data['lobby_num']=data['houseType'].apply(lobby_num)

#提取卫生间数量
def toilet_num(toilet):
    return int(toilet[4])

data['toilet_num']=data['houseType'].apply(toilet_num)

#填充租房类型
data.loc[(data['rentType']=='未知方式')&(data['room_num']<=1),'rentType']='整租'

data

,ID,area,rentType,houseType,houseFloor,totalFloor,houseToward,houseDecoration,communityName,city,...,uv,lookNum,tradeTime,tradeMoney,type,month,days,room_num,lobby_num,toilet_num
0,100309852,68.06,未知方式,2室1厅1卫,低,16,暂无数据,其他,XQ00051,SH,...,284.0,0,2018/11/28,2000.0,train,11,28,2,1,1
1,100307942,125.55,未知方式,3室2厅2卫,中,14,暂无数据,简装,XQ00130,SH,...,22.0,1,2018/12/16,2000.0,train,12,16,3,2,2
2,100307764,132.00,未知方式,3室2厅2卫,低,32,暂无数据,其他,XQ00179,SH,...,20.0,1,2018/12/22,16000.0,train,12,22,3,2,2
3,100306518,57.00,整租,1室1厅1卫,中,17,暂无数据,精装,XQ00313,SH,...,279.0,9,2018/12/21,1600.0,train,12,21,1,1,1
4,100305262,129.00,未知方式,3室2厅3卫,低,2,暂无数据,毛坯,XQ01257,SH,...,480.0,0,2018/11/18,2900.0,train,11,18,3,2,3
5,100304268,223.35,未知方式,4室3厅3卫,低,2,暂无数据,其他,XQ00457,SH,...,1304.0,0,2018/11/14,14000.0,train,11,14,4,3,3
6,100304055,78.94,未知方式,2室2厅1卫,中,18,暂无数据,其他,XQ00246,SH,...,469.0,0,2018/11/18,1000.0,train,11,18,2,2,1
7,100295123,76.00,未知方式,2室1厅1卫,中,5,暂无数据,其他,XQ02331,SH,...,2942.0,0,2018/3/24,1800.0,train,3,24,2,1,1
8,100293141,120.55,未知方式,3室2厅1卫,中,6,暂无数据,其他,XQ02346,SH,...,2942.0,0,2018/3/19,1450.0,train,3,19,3,2,1
9,100293063,83.00,未知方式,3室1厅1卫,中,5,暂无数据,其他,XQ02310,SH,...,3035.0,0,2018/4/26,1700.0,train,4,26,3,1,1


In [7]:
#对房间的类型进行特征交叉
#这里看出一室一厅更受欢迎

data['room_lobby']=data['room_num'].astype('str')+'_'+data['lobby_num'].astype('str')
data['room_toilet']=data['room_num'].astype('str')+'_'+data['toilet_num'].astype('str')
data['lobby_toilet']=data['lobby_num'].astype('str')+'_'+data['toilet_num'].astype('str')
print(data['room_lobby'].value_counts())
print(data['room_toilet'].value_counts())
print(data['room_toilet'].value_counts())

1_1    10430
2_1     9321
2_2     8222
3_2     5155
3_1     4241
4_1     2002
1_0     1453
1_2      998
4_2      810
2_0      473
5_1      206
5_2      126
4_3       85
6_1       66
5_3       63
6_2       29
3_3       29
3_0       24
7_1       18
6_3       10
5_4        6
7_2        6
4_0        6
2_3        4
6_4        3
3_4        1
4_4        1
8_3        1
7_4        1
9_2        1
7_3        1
5_5        1
8_4        1
8_2        1
6_0        1
Name: room_lobby, dtype: int64
2_1    16874
1_1    12718
3_1     6158
3_2     3106
4_1     2013
2_2     1116
4_2      487
4_3      301
5_1      213
3_3      167
1_0      103
4_4       94
5_3       80
6_1       68
1_2       60
5_4       47
5_2       45
2_0       22
7_1       18
6_4       18
5_5       16
3_4       15
4_5        9
6_3        8
2_3        8
6_2        7
7_3        4
6_5        4
3_0        3
6_7        3
7_4        3
8_4        2
6_6        1
3_5        1
8_3        1
5_6        1
9_5        1
7_7        1
Name: room_toilet, d

## 特征合并

In [8]:
def merge_(data):
    data['transport']=data['subwayStationNum']/data['subwayStationNum'].mean()+2*data['busStationNum']/data['busStationNum'].mean()
    data['school']=1.5*data['interSchoolNum']/data['interSchoolNum'].mean()+data['schoolNum']/data['schoolNum'].mean()\
                    +1.2*data['privateSchoolNum']/data['privateSchoolNum'].mean()
    data['hospital']=1.5*data['hospitalNum']/data['hospitalNum'].mean()+data['drugStoreNum']/data['drugStoreNum'].mean()
    data['mall']=1.5*data['mallNum']/data['mallNum'].mean()+data['superMarketNum']/data['superMarketNum'].mean()
    data['other']=1.5*data['gymNum']/data['gymNum'].mean()+1.8*data['bankNum']/data['bankNum'].mean()+\
                    data['shopNum']/data['shopNum'].mean()+data['parkNum']/data['parkNum'].mean()
    data.drop(['subwayStationNum', 'busStationNum', 'interSchoolNum', 'schoolNum', 'privateSchoolNum', 'hospitalNum', 'drugStoreNum',\
               'gymNum', 'bankNum', 'shopNum', 'parkNum', 'mallNum', 'superMarketNum'],inplace=True,axis=1)
    return data

data=merge_(data)

## 计算统计特征

In [9]:
#类别统计特征
def feature_count(data,features):
    new_feature='_count'
    for f in features:
        new_feature=f+new_feature
    temp=data.groupby(features).size().reset_index().rename(columns={0:new_feature})
    data=data.merge(temp,on=features,how='left')
    #data[new_feature].fillna(0)
    return data

data=feature_count(data,['communityName'])
data=feature_count(data,['totalFloor'])
data=feature_count(data,['buildYear'])
data=feature_count(data,['plate'])
data=feature_count(data,['communityName','totalFloor'])
data=feature_count(data,['communityName','buildYear'])
data=feature_count(data,['region','plate'])

In [10]:
#连续型计算特征
def group_count(data):
    #平均每平米房价
    data['ave_money']=data['tradeNewMeanPrice']/data['area']
    
    temp=data.groupby(['communityName'])['area'].agg({'community_mean_area':'mean','community_std_area':'std'})
    temp.fillna(0,inplace=True)
    data=data.merge(temp,on=['communityName'],how='left')
    
    temp=data.groupby(['communityName'])['ave_money'].agg({'community_mean_ave_money':'mean','community_std_ave_money':'std'})
    temp.fillna(0,inplace=True)
    data=data.merge(temp,on=['communityName'],how='left')
    
    temp=data.groupby(['plate'])['ave_money'].agg({'plate_mean_ave_money':'mean','plate_std_ave_money':'std'})
    temp.fillna(0,inplace=True)
    data=data.merge(temp,on=['plate'],how='left')
    
    temp=data.groupby(['plate'])['area'].agg({'plate_mean_area':'mean','plate_std_area':'std'})
    temp.fillna(0,inplace=True)
    data=data.merge(temp,on=['plate'],how='left')
    
    temp=data.groupby(['houseType'])['ave_money'].agg({'plate_mean_ave_money':'mean','plate_std_ave_money':'std'})
    temp.fillna(0,inplace=True)
    data=data.merge(temp,on=['houseType'],how='left')
    
    temp=data.groupby(['communityName'])['transport'].agg({'community_mean_transport':'mean','community_std_transport':'std'})
    temp.fillna(0,inplace=True)
    data=data.merge(temp,on=['communityName'],how='left')
    
    temp=data.groupby(['communityName'])['mall'].agg({'community_mean_mall':'mean','community_std_mall':'std'})
    temp.fillna(0,inplace=True)
    data=data.merge(temp,on=['communityName'],how='left')
    
    temp=data.groupby(['region'])['transport'].agg({'region_mean_transport':'mean','region_std_transport':'std'})
    temp.fillna(0,inplace=True)
    data=data.merge(temp,on=['region'],how='left')
    
    temp=data.groupby(['communityName','region'])['ave_money'].agg({'communityName_region_mean_ave_money':'mean','communityName_region_std_ave_money':'std'})
    temp.fillna(0,inplace=True)
    data=data.merge(temp,on=['communityName','region'],how='left')
    
    temp=data.groupby(['communityName','plate'])['ave_money'].agg({'communityName_plate_mean_ave_money':'mean','communityName_plate_std_ave_money':'std'})
    temp.fillna(0,inplace=True)
    data=data.merge(temp,on=['communityName','plate'],how='left')
    
    temp=data.groupby(['communityName','plate'])['transport'].agg({'communityName_plate_mean_transport':'mean','communityName_plate_std_transport':'std'})
    temp.fillna(0,inplace=True)
    data=data.merge(temp,on=['communityName','plate'],how='left')
    
    temp=data.groupby(['communityName'])['school'].agg({'community_mean_school':'mean','community_std_school':'std'})
    temp.fillna(0,inplace=True)
    data=data.merge(temp,on=['communityName'],how='left')
    
    temp=data.groupby(['plate'])['mall'].agg({'plate_mean_mall':'mean','plate_std_mall':'std'})
    temp.fillna(0,inplace=True)
    data=data.merge(temp,on=['plate'],how='left')
    
    temp=data.groupby(['plate'])['mall'].agg({'plate_mean_mall':'mean','plate_std_mall':'std'})
    temp.fillna(0,inplace=True)
    data=data.merge(temp,on=['plate'],how='left')
    
    temp=data.groupby(['plate'])['other'].agg({'plate_mean_other':'mean','plate_std_other':'std'})
    temp.fillna(0,inplace=True)
    data=data.merge(temp,on=['plate'],how='left')
    
    temp=data.groupby(['region','plate'])['hospital'].agg({'region_plate_mean_hospital':'mean','region_plate_std_hospital':'std'})
    temp.fillna(0,inplace=True)
    data=data.merge(temp,on=['region','plate'],how='left')
    
    temp=data.groupby(['buildYear'])['ave_money'].agg({'buildYear_mean_ave_money':'mean','buildYear_std_ave_money':'std'})
    temp.fillna(0,inplace=True)
    data=data.merge(temp,on=['buildYear'],how='left')
    return data

data=group_count(data)


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  # Remove the CWD from sys.path while we load stuff.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: using a dict on a Series for aggregation

In [18]:
data

,ID,area,rentType,houseType,houseFloor,totalFloor,houseToward,houseDecoration,communityName,city,...,plate_mean_mall_x,plate_std_mall_x,plate_mean_mall_y,plate_std_mall_y,plate_mean_other,plate_std_other,region_plate_mean_hospital,region_plate_std_hospital,buildYear_mean_ave_money,buildYear_std_ave_money
0,100309852,4.234976,2,12,1,2.833213,6,0,50,SH,...,0.685838,0.0,0.685838,0.0,1.133688,0.0,0.442397,0.0,7.026890,7.274377
1,100307942,4.840637,2,29,0,2.708050,6,2,126,SH,...,0.946093,0.0,0.946093,0.0,1.142683,0.0,0.597196,0.0,7.038933,7.374234
2,100307764,4.890349,2,29,1,3.496508,6,0,175,SH,...,0.842414,0.0,0.842414,0.0,1.582616,0.0,0.900500,0.0,7.061881,7.400871
3,100306518,4.060443,1,3,0,2.890372,6,3,309,SH,...,1.037538,0.0,1.037538,0.0,1.735262,0.0,1.653575,0.0,7.061881,7.400871
4,100305262,4.867534,2,30,1,1.098612,6,1,1251,SH,...,0.636834,0.0,0.636834,0.0,0.939573,0.0,0.311020,0.0,7.061881,7.400871
5,100304268,5.413207,2,50,1,1.098612,6,0,453,SH,...,1.401102,0.0,1.401102,0.0,2.465054,0.0,2.069575,0.0,7.035318,7.411318
6,100304055,4.381276,2,16,0,2.944439,6,0,242,SH,...,1.037538,0.0,1.037538,0.0,1.735262,0.0,1.653575,0.0,7.166790,7.390198
7,100295123,4.343805,2,12,0,1.791759,6,0,2314,SH,...,1.670757,0.0,1.670757,0.0,2.166974,0.0,1.595188,0.0,6.888040,7.388178
8,100293141,4.800326,2,28,0,1.945910,6,0,2329,SH,...,1.670757,0.0,1.670757,0.0,2.166974,0.0,1.595188,0.0,7.038933,7.374234
9,100293063,4.430817,2,24,0,1.791759,6,0,2293,SH,...,1.670757,0.0,1.670757,0.0,2.166974,0.0,1.595188,0.0,6.921039,7.235087


In [11]:

#根据分析，可以将数据属性分为类别特征和连续型特征
#类别特征
category_feauture=[ 'rentType', 'houseType', 'houseFloor','houseToward', 'houseDecoration', 'communityName', 'region',
                    'plate', 'buildYear', 'room_lobby', 'room_toilet', 'lobby_toilet']

#连续型特征
Value_feauture=[f for f in data.columns if f not in category_feauture and f not in ['ID','tradeTime','tradeMoney','type','city']]
print(Value_feauture)

target=['tradeMoney']

['area', 'totalFloor', 'saleSecHouseNum', 'totalTradeMoney', 'totalTradeArea', 'tradeMeanPrice', 'tradeSecNum', 'totalNewTradeMoney', 'totalNewTradeArea', 'tradeNewMeanPrice', 'tradeNewNum', 'remainNewNum', 'supplyNewNum', 'supplyLandNum', 'supplyLandArea', 'tradeLandNum', 'tradeLandArea', 'landTotalPrice', 'landMeanPrice', 'totalWorkers', 'newWorkers', 'residentPopulation', 'pv', 'uv', 'lookNum', 'month', 'days', 'room_num', 'lobby_num', 'toilet_num', 'transport', 'school', 'hospital', 'mall', 'other', 'communityName_count', 'totalFloor_count', 'buildYear_count', 'plate_count', 'totalFloorcommunityName_count', 'buildYearcommunityName_count', 'plateregion_count', 'ave_money', 'community_mean_area', 'community_std_area', 'community_mean_ave_money', 'community_std_ave_money', 'plate_mean_ave_money_x', 'plate_std_ave_money_x', 'plate_mean_area', 'plate_std_area', 'plate_mean_ave_money_y', 'plate_std_ave_money_y', 'community_mean_transport', 'community_std_transport', 'community_mean_mall'

In [12]:
for f in category_feauture:
    ln=LabelEncoder()
    data[f]=ln.fit_transform(data[f])
'''   
for f in Value_feauture:
    if f not in ['ave_money']:
        mms=MinMaxScaler(feature_range=(0,1))
        data[f]=mms.fit_transform(data[f].values.reshape(-1,1))''' 

"   \nfor f in Value_feauture:\n    if f not in ['ave_money']:\n        mms=MinMaxScaler(feature_range=(0,1))\n        data[f]=mms.fit_transform(data[f].values.reshape(-1,1))"

In [13]:
feas=category_feauture+Value_feauture

# 聚类特征

In [14]:
'''from sklearn.mixture import GaussianMixture
def cluster(data):
    cols=[ 'rentType', 'houseType', 'houseFloor','houseToward', 'houseDecoration', 'communityName', 'region',
          
         'plate', 'buildYear', 'room_lobby', 'room_toilet', 'lobby_toilet', 
          
        'totalFloor', 'saleSecHouseNum', 'totalTradeMoney', 'totalTradeArea', 'tradeMeanPrice', 'tradeSecNum', 
          
          'totalNewTradeMoney', 'totalNewTradeArea', 'tradeNewMeanPrice', 'tradeNewNum', 'remainNewNum', 'supplyNewNum',
          
          'supplyLandNum', 'supplyLandArea', 'tradeLandNum', 'tradeLandArea', 'landTotalPrice',
          
          'landMeanPrice', 'totalWorkers', 'newWorkers', 'residentPopulation', 'pv', 'uv', 'lookNum',
          
          'month', 'days', 'room_num', 'lobby_num', 'toilet_num', 'transport', 'school', 'hospital', 'mall',
          
          'other', 'communityName_count', 'totalFloor_count','buildYear_count', 'plate_count',
          
          'totalFloorcommunityName_count', 'buildYearcommunityName_count', 'plateregion_count']
    
    gmm=GaussianMixture(n_components=3, covariance_type='full', random_state=0)
    data['cluster']=pd.DataFrame(gmm.fit(data[cols]).predict(data[cols]))
    
    col1=['rentType', 'houseType', 'houseFloor','houseToward', 'houseDecoration', 'communityName', 'region',
          
         'plate', 'buildYear', 'room_lobby', 'room_toilet', 'lobby_toilet']
    
    col2=['totalFloor', 'saleSecHouseNum', 'totalTradeMoney', 'totalTradeArea', 'tradeMeanPrice', 'tradeSecNum', 
          
          'totalNewTradeMoney', 'totalNewTradeArea', 'tradeNewMeanPrice', 'tradeNewNum', 'remainNewNum', 'supplyNewNum',
          
          'supplyLandNum', 'supplyLandArea', 'tradeLandNum', 'tradeLandArea', 'landTotalPrice',
          
          'landMeanPrice', 'totalWorkers', 'newWorkers', 'residentPopulation', 'pv', 'uv', 'lookNum',
          
          'month', 'days', 'room_num', 'lobby_num', 'toilet_num', 'transport', 'school', 'hospital', 'mall',
          
          'other', 'communityName_count', 'totalFloor_count','buildYear_count', 'plate_count',
          
          'totalFloorcommunityName_count', 'buildYearcommunityName_count', 'plateregion_count']
    
    for f1 in col1:
        for f2 in col2:
            temp=data.groupby(['cluster',f1])[f2].agg('mean').reset_index(name=f1+'feature'+f2)
            temp.fillna(0,inplace=True)
            data=data.merge(temp,on=['cluster',f1],how='left')
            
    return data

data=cluster(data)
#连续型特征
Value_feauture=[f for f in data.columns if f not in category_feauture and f not in ['ID','tradeTime','tradeMoney','type','city']]

print(Value_feauture)
'''
#做log平滑，与LabelEncoder 相比，log平滑线下下降，但线上有提高

'''for f in Value_feauture:
    if f not in ['ave_money']:
        mms=MinMaxScaler(feature_range=(0,1))
        data[f]=mms.fit_transform(data[f].values.reshape(-1,1))
'''
for f in Value_feauture:
    data[f]=data[f].map(lambda x:np.log1p(x))
    
feas=category_feauture+Value_feauture

In [15]:
#划分数据集

train_=data[data['type']=='train'][feas]
test_=data[data['type']=='test'][feas]
train_target=data[data['type']=='train']['tradeMoney']
test_target=data[data['type']=='test']['tradeMoney']

#划分数据为训练集和验证集，比例为 8:2

train_x,valid_x,train_y,valid_y=train_test_split(train_,train_target,test_size=0.2,random_state=42)


In [16]:
#线性模型测试特征工程效果
from sklearn.linear_model import Lasso
lasso=Lasso(alpha=0.1)
lasso.fit(train_x,train_y)
pre_train=lasso.predict(train_x)
pre_valid=lasso.predict(valid_x)

#对比结果
r2=r2_score(pre_train,train_y)
print("训练集结果:{}".format(r2))
r2=r2_score(pre_valid,valid_y)
print("验证集结果:{}".format(r2))

训练集结果:0.4221775196687515
验证集结果:0.3476366464016758


D:\Anaconda\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


# 特征选择

## 相关系数法

In [17]:
from sklearn.feature_selection import SelectKBest

print(train_x.shape)
sk=SelectKBest(k=300)
new_train=sk.fit_transform(train_x,train_y)

print(new_train.shape)

#获取对应的列索引
select_columns=sk.get_support(indices=True)
print(select_columns)
columns_name=valid_x.columns[select_columns]

print(columns_name)
new_valid=valid_x[columns_name]

#线性模型测试效果
from sklearn.linear_model import Lasso
lasso=Lasso(alpha=0.1)
lasso.fit(new_train,train_y)
pre_train=lasso.predict(new_train)
pre_valid=lasso.predict(new_valid)

#对比结果
r2=r2_score(pre_train,train_y)
print("训练集结果:{}".format(r2))
r2=r2_score(pre_valid,valid_y)
print("验证集结果:{}".format(r2))

(33061, 89)


ValueError: k should be >=0, <= n_features; got 300.Use k='all' to return all features.

## Wrapper

In [ ]:
# 递归消除特征法使用一个基模型来进行多轮训练，每轮训练后，消除若干权值系数的特征，
# 再基于新的特征集进行下一轮训练。使用feature_selection库的RFE类来选择特征的代码如下：

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

#递归特征消除法，返回特征选择后的数据
#参数estimator为基模型
#参数n_features_to_select为选择的特征个数

lr=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                               normalize=False)
ref=RFE(lr,n_features_to_select=300)
ref.fit(train_x,train_y)

#REF(estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
#                               normalize=False),
#    n_features_to_select=200,step=1,verbose=0)

#获取列索引
select_columns=[f for f,s in zip(train_x.columns,ref.support_) if s]

print(select_columns)

new_train=train_x[select_columns]
new_valid=valid_x[select_columns]

#测试效果

from sklearn.linear_model import Lasso
lasso=Lasso(alpha=0.1)
lasso.fit(new_train,train_y)
pre_train=lasso.predict(new_train)
pre_valid=lasso.predict(new_valid)

#对比结果
r2=r2_score(pre_train,train_y)
print("训练集结果:{}".format(r2))
r2=r2_score(pre_valid,valid_y)
print("验证集结果:{}".format(r2))

# Embedded
### 基于惩罚项的特征选择方法（l1和l2）

Lasso(l1)和Ridge(l2)

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

ridge=Ridge(alpha=5)
ridge.fit(train_x,train_y)
Ridge(alpha=5,copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)


#特征系数排序
coefSort=ridge.coef_.argsort()
print(coefSort)

#特征系数
FeatureSoce=ridge.coef_[coefSort]
print(FeatureSoce)

select_columns=[f for f,s in zip(train_x.columns,FeatureSoce) if abs(s)> 1 ]

new_train=train_x[select_columns]
new_valid=valid_x[select_columns]
#测试效果

from sklearn.linear_model import Lasso
lasso=Lasso(alpha=0.1)
lasso.fit(new_train,train_y)
pre_train=lasso.predict(new_train)
pre_valid=lasso.predict(new_valid)

#对比结果
r2=r2_score(pre_train,train_y)
print("训练集结果:{}".format(r2))
r2=r2_score(pre_valid,valid_y)
print("验证集结果:{}".format(r2))

### 基于树模型的特征选择

随即森林 平均不纯度减少

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf=RandomForestRegressor()
rf.fit(train_x,train_y)
# 训练随机森林模型，并通过feature_importances_属性获取每个特征的重要性分数。rf = RandomForestRegressor()

print("Features sorted by their score:")
print(sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), train_x.columns),
             reverse=True))

select_columns=[f for f ,s in zip(train_x.columns,rf.feature_importances_) if abs(s)>0.0001]
print(select_columns)

new_train=train_x[select_columns]
new_valid=valid_x[select_columns]
#测试效果

from sklearn.linear_model import Lasso
lasso=Lasso(alpha=0.1)
lasso.fit(new_train,train_y)
pre_train=lasso.predict(new_train)
pre_valid=lasso.predict(new_valid)

#对比结果
r2=r2_score(pre_train,train_y)
print("训练集结果:{}".format(r2))
r2=r2_score(pre_valid,valid_y)
print("验证集结果:{}".format(r2))

In [ ]:
,,,
#构建baseline模型
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'num_leaves': 63,
    'learning_rate': 0.01,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_seed':0,
    'bagging_freq': 1,
    'verbose': 1,
    'reg_alpha':1,
    'reg_lambda':2,
    'min_child_weight':6
}

lgb_train=lgb.Dataset(train_x,train_y)
lgb_eval=lgb.Dataset(valid_x,valid_y,reference=lgb_train)

gbm=lgb.train(params,lgb_train,num_boost_round=10000,valid_sets=[lgb_train,lgb_eval],
                valid_names=['train','valid'],
                early_stopping_rounds=2000,
                verbose_eval=1000,
                )
valid=gbm.predict(valid_x)
feature_rank=gbm.feature_importance()
features_df=pd.DataFrame({ 'column':feas,'importance': feature_rank}).sort_values(by='importance', ascending=False)
print(features_df)

In [ ]:
from sklearn.metrics import r2_score
r2=r2_score(valid,valid_y)
print("验证集结果:{}".format(r2))

# 未做特征工程前，验证集结果 :0.8480048499945494

#做特征工程，没加方差和均值统计特征前，验证集结果:0.8569968502829308（线下有提高）

#加了方差和均值统计特征，验证集结果:0.8669673740210536（有提高）

#做了聚类特征 验证集结果:0.8654237103479906，线下有点下降

In [ ]:
'''val=valid_x[['area']]
val['pred']=valid
val['pre']=valid_y
def score(real_data):
    a=sum((real_data['pred']-real_data['pre'])*(real_data['pred']-real_data['pre']))
    b=sum((real_data['pre']-real_data['pre'].mean())*(real_data['pre']-real_data['pre'].mean()))
    s=1-a/b
    return s

val_s=score(val)
val_s
# 0.8654572967842439

#做了归一化之后 0.8661834939412152
'''

In [ ]:
#训练整个数据模型，并预测
lgb_train_=lgb.Dataset(train_,train_target)
gbm=lgb.train(params,lgb_train_,num_boost_round=10000,valid_sets=[lgb_train_],
              valid_names=['train'],
              early_stopping_rounds=2000,
              verbose_eval=1000)

real_data['pred']=gbm.predict(test_)

###  测试结果记录

#未做很多的特征工程，只是补缺失值，然后将类别特征进行了处理得到结果：  0.40585301363216797

#删除异常数据后，进行预测结果：0.9211771459530279 ，这只是清除了异常数据就这么高???????是不是有点假??????

#做了归一化之后 结果是：0.9214346671924497

#做了特征工程后，没加方差和均值统计特征前：0.9433749685395988（有提高）

#加方差和均值统计特征 0.94636263988483（有些许提高）

#加了聚类特征，0.9279984686417421(线上也降低了)

In [ ]:
r2=r2_score(real_data['pred'],real_data['pre'])
print(r2)